# Interval method for weather predictions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import mean_squared_error

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 12)

In [4]:
def relative_rmse(target, preds):
    rmse = mean_squared_error(target, preds, squared=False)
    avg = np.full(len(preds), preds.mean())
    const_rmse = mean_squared_error(target, avg, squared=False)
    return rmse / const_rmse

### Reading data

In [13]:
high = pd.read_csv('predicted_weather/prophet_high.csv')
low = pd.read_csv('predicted_weather/prophet_low.csv')

In [14]:
low['VAP'][low['VAP'] < 0] = 0.06
low['RAIN'][low['RAIN'] < 0] = 0

/var/folders/8j/21yjmtfs3h5bg4fw61rbh_rr0000gn/T/ipykernel_8223/844755778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low['VAP'][low['VAP'] < 0] = 0.06
/var/folders/8j/21yjmtfs3h5bg4fw61rbh_rr0000gn/T/ipykernel_8223/844755778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low['RAIN'][low['RAIN'] < 0] = 0


In [16]:
high['VAP'][high['VAP'] < 0] = 0.06
high['RAIN'][high['RAIN'] < 0] = 0

/var/folders/8j/21yjmtfs3h5bg4fw61rbh_rr0000gn/T/ipykernel_8223/2604046005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high['VAP'][high['VAP'] < 0] = 0.06
/var/folders/8j/21yjmtfs3h5bg4fw61rbh_rr0000gn/T/ipykernel_8223/2604046005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high['RAIN'][high['RAIN'] < 0] = 0


In [17]:
low.to_csv('predicted_weather/prophet_low.csv', index=False)
high.to_csv('predicted_weather/prophet_high.csv', index=False)

In [18]:
pivot = low.merge(high, on=['DAY', 'SNOWDEPTH'])

In [19]:
def get_dateset(df, name, n):
    df[f'Width_{name}'] = df[f'{name}_y'] - df[f'{name}_x'] 
    df[f'Delta_{name}'] = df[f'Width_{name}'] / n
    temp = df[[f'Delta_{name}', f'{name}_x']].copy()
    for i in range(1, n+1):
        temp[f'{name}_{i}'] = temp[f'{name}_x'] + i * temp[f'Delta_{name}']
    temp.rename({f'{name}_x': f'{name}_0'}, axis=1, inplace=True)
    return temp.drop([f'Delta_{name}'], axis=1)

In [20]:
n = 8

In [21]:
irrad = get_dateset(pivot, 'IRRAD', n)
tmin = get_dateset(pivot, 'TMIN', n)
tmax = get_dateset(pivot, 'TMAX', n)
vap = get_dateset(pivot, 'VAP', n)
wind = get_dateset(pivot, 'WIND', n)
rain = get_dateset(pivot, 'RAIN', n)

In [22]:
irrad.to_csv('predicted_weather/interval_data/irrad.csv', index=False)
tmin.to_csv('predicted_weather/interval_data/tmin.csv', index=False)
tmax.to_csv('predicted_weather/interval_data/tmax.csv', index=False)
vap.to_csv('predicted_weather/interval_data/vap.csv', index=False)
wind.to_csv('predicted_weather/interval_data/wind.csv', index=False)
rain.to_csv('predicted_weather/interval_data/rain.csv', index=False)

In [ ]:
def product_columns():
    for x1 in range(n+1):
        for x2 in range(n+1):
            for x3 in range(n+1):
                for x4 in range(n+1):
                    for x5 in range(n+1):
                        for x6 in range(n+1):
                            tmp = low[['DAY', 'SNOWDEPTH']].copy()
                            tmp['IRRAD'] = irrad[f'IRRAD_{x1}']
                            tmp['TMIN'] = tmin[f'TMIN_{x2}']
                            tmp['TMAX'] = tmax[f'TMAX_{x3}']
                            tmp['VAP'] = vap[f'VAP_{x4}']
                            tmp['WIND'] = wind[f'WIND_{x5}']
                            tmp['RAIN'] = rain[f'RAIN_{x6}']
                            tmp.to_csv(f'predicted_weather/interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv', index=False)

In [ ]:
product_columns()